In [187]:
'''
Decision Tree 
Li Teng
27.03.2020
'''

import pandas as pd
import numpy as np
import math

def get_true_label(arr):
    #quality >= 6 as good(1),<6 as bad(-1)
    arr = np.sign(arr-5.5)
    return arr

def calc_prob(Data):
    #comput the positive & negative prob of the data
    positive = 0
    negative = 0
    for d in Data:
        if d[11] == 1:
            positive +=1
        else:
            negative +=1
    positive_prob = positive/(positive+negative)
    negative_prob = negative/(positive+negative)
    return positive_prob, negative_prob

def get_class(pos_prob,neg_prob):
    if pos_prob>0.99:
        C = 1
    elif neg_prob>0.99:
        C = -1
    else:
        C = None
    return C

def esti_class(pos_prob,neg_prob):
    if pos_prob>neg_prob:
        C = 1
    else:
        C = -1
    return C

def get_Entropy(Data):
    #comput the entropy of the data
    positive_prob, negative_prob = calc_prob(Data)
    #print('pos:',positive_prob)
    #print('neg:',negative_prob)
    if positive_prob == 0:
        #print('pure negative')
        Entropy = 0
    elif negative_prob == 0:
        #print('pure positive')
        Entropy = 0
    else:
        Entropy = -(positive_prob*math.log(positive_prob,2)+negative_prob*math.log(negative_prob,2))
    return Entropy

def find_Split_values(Data,Feature):
    #find all split values of the given Feature
    All_values = list(set(Data[:,Feature]))
    All_values.sort()
    All_Split_values = []
    for index in range(len(All_values)-1):
        All_Split_values.append((All_values[index]+All_values[index+1])/2)
    return All_Split_values

def split_data(Data,Feature,Split_value):
    #split data into 2 subdatas
    lenth = 0
    for d in Data:
        if d[Feature] < Split_value:
            lenth+=1
        else:
            break            
    SubData1 = Data[:lenth,:]
    SubData2 = Data[lenth:,:]
    return SubData1, SubData2
    
def find_best_Split_values(Data,Feature):
    #find best split values of the given Feature from All_split_values
    data = Data[Data[:,Feature].argsort()]
    Best_Data1 = None
    Best_Data2 = None
    Best_Entropy = float('inf')
    Best_Value = 0
    Data_entities = Data.shape[0]
    All_Split_values = find_Split_values(Data,Feature)
    for value in All_Split_values:
        #print("split value:",value)
        SubData1, SubData2 = split_data(data,Feature,value)
        Entropy = (SubData1.shape[0]*get_Entropy(SubData1) + SubData2.shape[0]*get_Entropy(SubData2))/Data_entities
        if Entropy < Best_Entropy:
            Best_Entropy = Entropy
            Best_Value = value
            Best_Data1 = SubData1
            Best_Data2 = SubData2
    #print("Feature:",Feature,"Value",Best_Value)
    return Best_Data1, Best_Data2, Best_Value, Best_Entropy

def get_best_feature(Data):
    #find the best feature according to the Gain
    Best_Feature = None
    Best_Split_value = None
    Best_Entropy = float('inf')
    Data1 = None
    Data2 = None
    for F in range(10):
        #print('Feature:',F)
        D1,D2,Split_Value,Entropy = find_best_Split_values(Data,F)
        if Entropy < Best_Entropy:
            #print('replace')
            Best_Feature = F
            Best_Entropy = Entropy
            Best_Split_value = Split_Value
            Data1 = D1
            Data2 = D2
    return Best_Feature, Data1, Data2, Best_Split_value

class Node:
    def __init__(self,Leaf,split_value,Feature,lchild=None,rchild=None,C=None):
        #self.Dataset = Dataset
        self.Leaf = Leaf
        self.Feature = Feature
        self.split_value = split_value
        self.C = C
        self.lchild = lchild
        self.rchild = rchild

class Tree:
    def __init__(self,First_split_value,First_split_feature):
        self.root = Node(False,First_split_value,First_split_feature)
        self.queue = []
        self.queue.append(self.root)
    
    def add(self,value,feature,C):
        #print('add C:',C)
        Tree_node = self.queue[0]
        if C == 1 or C == -1:
            #this node should be a leaf
            #print('leaf')
            New_node = Node(True,None,None,None,None,C)
            if Tree_node.lchild == None:
                Tree_node.lchild = New_node
            else:
                Tree_node.rchild = New_node
                self.queue.pop(0)
        else:
            #this is not a leaf
            New_node = Node(False,value,feature)
            if Tree_node.lchild == None:
                Tree_node.lchild = New_node
                self.queue.append(New_node)
            else:
                Tree_node.rchild = New_node
                self.queue.append(New_node)
                self.queue.pop(0)
                
    def predict(self,root,Data):
        if root.Leaf == True:
            #this is leaf node
            predict_label = root.C
            return predict_label
        else:
            if Data[root.Feature] < root.split_value:
                return self.predict(root.lchild,Data)
            else:
                return self.predict(root.rchild,Data)
            
    def depth_first_search(self,root):
        if root == None:
            return None
        else:
            print(root.Leaf,root.C)
            self.depth_first_search(root.lchild)
            self.depth_first_search(root.rchild)


#Data preprocessing 
data = pd.read_csv('/home/teng/Github/Decision_Tree/winequality-red.csv')
data = data.as_matrix(columns=None)
np.random.shuffle(data)
Training_Set, Test_Set = data[:1499,:], data[1499:,:]
#Col 0-10 are Features and Col 11 is label.
Training_Set[:,11] = get_true_label(Training_Set[:,11])
Test_Set[:,11] = get_true_label(Test_Set[:,11])

Root_Feature, Data1, Data2, Root_Split_value = get_best_feature(Training_Set)
#print(Root_Feature, Root_Split_value)
#use a queue to store the Data which is not 'pure'
queue = []
queue.append(Data1)
queue.append(Data2)
#construct the tree
Dec_Tree = Tree(Root_Split_value,Root_Feature)
while queue:
    Data = queue[0]
    positive_prob, negative_prob = calc_prob(Data)
    if Data.shape[0]<2:
        #this Dataset is too small
        C = esti_class(positive_prob,negative_prob)
    else:
        C = get_class(positive_prob,negative_prob)
    #print(' ')
    #print('Data:',Data.shape,' C:',C)
    #print('p_prob:',positive_prob,' n_prob:',negative_prob)
    if C==1 or C==-1:
        #pure pos or neg
        Dec_Tree.add(None,None,C)
        queue.pop(0)
    else:
        #not pure
        Feature, D1, D2, Split_value = get_best_feature(Data)
        queue.append(D1)
        queue.append(D2)
        Dec_Tree.add(Split_value,Feature,None)
        queue.pop(0)
print('The construct of Decision Tree is done!')

correct_instence = 0
wrong_instence = 0
for ins in Test_Set:
    predict_label = Dec_Tree.predict(Dec_Tree.root,ins)
    True_label = ins[11]
    if predict_label == True_label:
        correct_instence += 1
    else:
        wrong_instence += 1
acc = correct_instence/(correct_instence+wrong_instence)
print('test_acc:',acc)

correct_instence = 0
wrong_instence = 0
for ins in Training_Set:
    predict_label = Dec_Tree.predict(Dec_Tree.root,ins)
    True_label = ins[11]
    #print(predict_label, True_label)
    if predict_label == True_label:
        correct_instence += 1
    else:
        wrong_instence += 1
acc = correct_instence/(correct_instence+wrong_instence)
print('train_acc:',acc)

/home/teng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:182: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


The construct of Decision Tree is done!
test_acc: 0.81
train_acc: 1.0
